## MySQL connector 설치

In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 53.8 MB/s eta 0:00:00


## 삽입하고자 하는 데이터 확인 : 한경 뉴스

In [4]:
import pandas as pd

df = pd.read_csv('/content/han.csv')

In [5]:
df.head()

,date,url,content
0,2005.03.10,https://www.hankyung.com/article/2005030994811,산업단지 안에 5백평 미만의 작은 공장을 지을 수 있게 된다.\n\n최홍건 중소기업...
1,2005.03.10,https://www.hankyung.com/article/2005030998281,채권시장은 9일 만기별로 금리 등락이 엇갈렸다.\n\n지표금리인 3년짜리 국고채 수...
2,2005.03.10,https://www.hankyung.com/article/2005030997551,< 윤재현 세종증권 리서치센터장 >\n\n원자재가격 상승에 대한 우려의 목소리가 커...
3,2005.03.10,https://www.hankyung.com/article/2005030981866,피데스증권은 원자재 시장에서 투기 수요가 이탈하는 과정에서 증시와 환율시장에 미치는...
4,2005.03.10,https://www.hankyung.com/article/2005030994021,'헤지펀드와 투자은행의 죽음의 키스?' \n\n최근 대형 투자은행들의 헤지펀드에 대...


## csv의 charset확인

In [16]:
!file -i /content/han.csv

/content/han.csv: text/csv; charset=utf-8


## DB에 여러가지 문자셋 호환 여부 확인

In [17]:
import pandas as pd
import mysql.connector

# 데이터베이스 연결
conn = mysql.connector.connect(
    host=' ', # 엔드 포인트
    port=3306,
    user=' ', # 유저명
    password=' ', # 비밀번호
    database='team2', # 접근하고자 하는 데이터베이스 명
    charset='utf8mb4'  # 문자 집합 설정
)
cursor = conn.cursor()

# 테스트 데이터 준비
test_data = [
    {'date': '2024-01-01', 'content': '테스트 기사: 다양한 문자와 기호 포함! ✔️🚀✨'},
    {'date': '2024-01-02', 'content': '기본 ASCII 문자: ABCDEFGHIJKLMNOPQRSTUVWXYZ'},
    {'date': '2024-01-03', 'content': '특수 문자: ~!@#$%^&*()_+[]{}|;:\'",.<>?'},
    {'date': '2024-01-04', 'content': '다국어 문자: 中文, 日本語, 한국어, Русский, العربية'}
]

# 테스트 데이터를 데이터베이스에 삽입
for data in test_data:
    cursor.execute(
        "INSERT INTO team2.han_news (date, content) VALUES (%s, %s)",
        (data['date'], data['content'])
    )

conn.commit()

# 데이터베이스에서 삽입된 데이터 확인
cursor.execute("SELECT date, content FROM team2.han_news")
rows = cursor.fetchall()

print("데이터베이스에 삽입된 데이터:")
for row in rows:
    print(row)

# 연결 종료
cursor.close()
conn.close()


데이터베이스에 삽입된 데이터:
(datetime.date(2024, 1, 1), '테스트 기사: 다양한 문자와 기호 포함! ✔️🚀✨')
(datetime.date(2024, 1, 2), '기본 ASCII 문자: ABCDEFGHIJKLMNOPQRSTUVWXYZ')
(datetime.date(2024, 1, 3), '특수 문자: ~!@#$%^&*()_+[]{}|;:\'",.<>?')
(datetime.date(2024, 1, 4), '다국어 문자: 中文, 日本語, 한국어, Русский, العربية')


## 데이터 삽입

In [4]:
import pandas as pd
import mysql.connector
# 104,000개 가량의 뉴스 데이터를 삽입하는데에 약 3시간이 소요되었습니다.

# CSV 파일 읽기 (필요한 열만 선택)
df = pd.read_csv('/content/han.csv', encoding='utf-8', usecols=['date', 'content'])

# 날짜 형식 변환
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')

# MySQL 데이터베이스 연결
conn = mysql.connector.connect(
    host=' ', # 엔드 포인트
    port=3306,
    user='', # 유저명
    password=' ', # 비밀번호
    database='team2',
    charset='utf8mb4'  # 문자 집합 설정
)
cursor = conn.cursor()

# 데이터 삽입
error_rows = []
for index, row in df.iterrows():
    try:
        cursor.execute(
            "INSERT INTO team2.han_news (date, content) VALUES (%s, %s)",
            (row['date'], row['content'])
        )
    except mysql.connector.Error as err:
        error_rows.append((index, str(err)))
        print(f"오류 발생 행 {index}: {row['content']}")
        print(f"오류 메시지: {err}")

# 커밋 (문제가 없는 행만 커밋됨)
conn.commit()

# 오류가 발생한 행 정보 출력
if error_rows:
    print("\n오류가 발생한 행 목록:")
    for idx, error in error_rows:
        print(f"행 {idx}: {error}")

# 데이터베이스에서 삽입된 데이터 확인
cursor.execute("SELECT date, content FROM team2.han_news ORDER BY date DESC LIMIT 10")
rows = cursor.fetchall()

print("\n최근 10개 데이터베이스 데이터:")
for row in rows:
    print(row)

# 연결 종료
cursor.close()
conn.close()

오류 발생 행 34420: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 74914: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 74923: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 78761: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 78868: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 78898: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 79657: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 79795: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 80421: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 80445: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 81176: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 81274: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in 'field list'
오류 발생 행 81672: nan
오류 메시지: 1054 (42S22): Unknown column 'nan' in